# Adding the test data to our database

In [1]:
import sqlite3

In [50]:
conn = sqlite3.connect("../database/db.sql")
cur = conn.cursor()

In [20]:
import pandas as pd

df = pd.read_csv("/Users/adrianselk/Desktop/DHBW-Projekte/magumba/assets/test_questions.csv")

In [21]:
df.head()

,question,answer1,answer2,answer3,answer4,correct_answers,points
0,Was ist ein großes Risiko des Klimawandels für...,Erhöhte Belastung durch hitzebedingte Krankheiten,Unterbrechung der Gesundheitsdienste durch ext...,Verbesserte Luftqualität,Sinkende Krankheitsraten,"[""Erhöhte Belastung durch hitzebedingte Krankh...",3
1,Welche Bevölkerungsgruppe ist am meisten vom K...,Junge Erwachsene,Kinder,Ältere Menschen,Gesunde Menschen mittleren Alters,"[""Kinder"", ""Ältere Menschen""]",2
2,"Wie können Einzelpersonen dazu beitragen, die ...",Öffentliche Verkehrsmittel nutzen,Weniger Fleisch konsumieren,Klimaanlagen intensiver nutzen,Mehr Auto fahren,"[""Öffentliche Verkehrsmittel nutzen"", ""Weniger...",2
3,Welches Gesundheitsproblem steht in direktem Z...,Herzkrankheiten,Atemwegserkrankungen,Erhöhte Inzidenz von Infektionskrankheiten,Fettleibigkeit,"[""Atemwegserkrankungen"", ""Erhöhte Inzidenz von...",1
4,Welches der folgenden Ziele ist ein Nachhaltig...,Bezahlbare und saubere Energie,Kein Hunger,Maximaler Ressourcenverbrauch,Unbegrenztes Wirtschaftswachstum,"[""Bezahlbare und saubere Energie"", ""Kein Hunger""]",3


In [39]:
# Function to clear the Question and Answer tables
def clear_tables():
    cur.execute("DELETE FROM Answer")
    cur.execute("DELETE FROM Question")
    conn.commit()
clear_tables()


In [41]:
import csv
import uuid
import ast

# Function to insert data from CSV into the database
def insert_data_from_csv(csv_file):
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        
        for row in reader:
            # generate a unique ID for the question
            question_id = str(uuid.uuid4())
            question_text = row['question']
            points = int(row['points'])
            
            # insert the question into the Question table
            cur.execute("INSERT INTO Question (id, question_text, points) VALUES (?, ?, ?)",
                        (question_id, question_text, points))
            
            # extract answers and the correct answers list
            answers = [row['answer1'], row['answer2'], row['answer3'], row['answer4']]
            correct_answers = ast.literal_eval(row['correct_answers'])  # Convert correct_answers string to list
            
            # insert each answer into the Answer table
            for answer in answers:
                answer_id = str(uuid.uuid4())  # generate unique ID for each answer
                
                # determine if the current answer is correct
                is_correct = answer in correct_answers
                
                # Insert the answer into the Answer table
                cur.execute("INSERT INTO Answer (id, question_id, answer, is_correct, explanation) VALUES (?, ?, ?, ?, ?)",
                            (answer_id, question_id, answer, is_correct, ""))  # empty explanation for now
            
        conn.commit()

csv_file = '/Users/adrianselk/Desktop/DHBW-Projekte/magumba/assets/test_questions.csv'
insert_data_from_csv(csv_file)



In [42]:
# testing how Answer table looks like
cur.execute("""
    SELECT *
    FROM ANSWER
    LIMIT 20
""")
rows = cur.fetchall()

for row in rows:
    print(row)


('8b39033f-9201-4e9b-9bbc-918f2660d345', '8e4c7047-400f-4632-a5f4-26f52677eda6', 'Erhöhte Belastung durch hitzebedingte Krankheiten', 1, '')
('2c421d28-142a-433b-b1bf-5579ade26b7c', '8e4c7047-400f-4632-a5f4-26f52677eda6', 'Unterbrechung der Gesundheitsdienste durch extreme Wetterereignisse', 1, '')
('fb46e2d7-0e84-4439-bf34-238f4d41ac3d', '8e4c7047-400f-4632-a5f4-26f52677eda6', 'Verbesserte Luftqualität', 0, '')
('ad50d999-c62c-4877-82e9-b035fb9937f8', '8e4c7047-400f-4632-a5f4-26f52677eda6', 'Sinkende Krankheitsraten', 0, '')
('9ebaca15-dcb2-42f0-a025-93c377e6ce85', 'd51e85d5-00ed-43c6-a629-d7415ec0cb16', 'Junge Erwachsene', 0, '')
('43e291b7-193a-4e09-93e9-e109f764cfc7', 'd51e85d5-00ed-43c6-a629-d7415ec0cb16', 'Kinder', 1, '')
('a7d1e132-7e69-445f-b8ad-8724ae95434b', 'd51e85d5-00ed-43c6-a629-d7415ec0cb16', 'Ältere Menschen', 1, '')
('2d874878-5d01-4333-83de-345fb2f060c9', 'd51e85d5-00ed-43c6-a629-d7415ec0cb16', 'Gesunde Menschen mittleren Alters', 0, '')
('6e852fdf-7457-48fb-98ee-04fd

In [47]:
# testing how Question table looks like
cur.execute("""
    SELECT *
    FROM Question
    LIMIT 5
""")
rows = cur.fetchall()

for row in rows:
    print(row)

('8e4c7047-400f-4632-a5f4-26f52677eda6', 'Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 3)
('d51e85d5-00ed-43c6-a629-d7415ec0cb16', 'Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 2)
('604bc4e2-73e1-4858-9914-0c87b6a7b56a', 'Wie können Einzelpersonen dazu beitragen, die globale Erwärmung zu reduzieren?', 2)
('74bf5892-aede-49bf-b6ab-9e4afa571b15', 'Welches Gesundheitsproblem steht in direktem Zusammenhang mit dem Klimawandel?', 1)
('89ff5587-b543-4d56-b10a-85d275060376', 'Welches der folgenden Ziele ist ein Nachhaltigkeitsziel (SDG)?', 3)


In [51]:
# Fetch questions and their associated answers
cur.execute("""
    SELECT q.question_text, a.answer, a.is_correct
    FROM Question q
    JOIN Answer a ON q.id = a.question_id
    LIMIT 12
""")
rows = cur.fetchall()

for row in rows:
    print(row)

conn.close()


('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Erhöhte Belastung durch hitzebedingte Krankheiten', 1)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Unterbrechung der Gesundheitsdienste durch extreme Wetterereignisse', 1)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Verbesserte Luftqualität', 0)
('Was ist ein großes Risiko des Klimawandels für Gesundheitssysteme?', 'Sinkende Krankheitsraten', 0)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Junge Erwachsene', 0)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Kinder', 1)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Ältere Menschen', 1)
('Welche Bevölkerungsgruppe ist am meisten vom Klimawandel betroffen?', 'Gesunde Menschen mittleren Alters', 0)
('Wie können Einzelpersonen dazu beitragen, die globale Erwärmung zu reduzieren?', 'Öffentliche Verkehrsmittel nutzen', 1)
('Wie können Ei

In [52]:
conn.close()